# Whisper

This notebook covers how to use whisper from openAI, build an audio chain and deploy it in combination with llm to summarize a transcript gathered from an audio file. A translation task is also shown.

In [1]:
from langchain.audio_models import Whisper
from langchain.chains import AudioChain, LLMChain, SimpleSequentialChain
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate

# Transcript task

### Defining the audio model

A `Whisper` objects has the following arguments:

* `model_key` (str): model endpoint to use;
* `prompt` (optional[str]): openAi's whisper allows prompt to make the model context-aware and better transcribe/translate words;
* `language` (optional[str]): to add only in the case of transcript as a task;
* `max_chars` (optional[int]): max number of chars to return;
* `model` (optional[str]): name of the model to use, set to `whisper-1` by default;
* `temperature` (Optional[float]): temperature;
* `response format` (optional[str]): response format, set by default to `json`

An `AudioChain` object has the following arguments:
* `audio_model` (AudioBase): the audio model to use;
* `output_key` (str): the task to be performed by the model. Not all the audio models support all the tasks. In the case of Whisper, the possible tasks are `"transcribe"` and `"translate"`.


In [2]:
audio_model = Whisper(model_key="YOUR_OPENAI_KEY", max_chars=20000)
audio_chain = AudioChain(audio_model=audio_model, output_key="transcribe")

### Setting up llm and an LLMChain

In [3]:
llm = OpenAI(
    openai_api_key="YOUR_OPENAI_KEY",
    temperature=0.7,
)

template = """Speech: {text}
Write a short 3 sentence summary of the speech.
Summary:"""

prompt_template = PromptTemplate(input_variables=["text"], template=template)

summary_chain = LLMChain(llm=llm, prompt=prompt_template)

### Defining a SimpleSequentialChain and running

In [ ]:
speech_to_summary_chain = SimpleSequentialChain(
    chains=[audio_chain, summary_chain], verbose=True
)

In [5]:
audio_path = "example_data/doit.mp3"
speech_to_summary_chain.run(audio_path)



> Entering new SimpleSequentialChain chain...
Do it! Just do it! Don't let your dreams be dreams. Yesterday, you said tomorrow. So just do it! Make your dreams come true! Just do it! Some people dream of success while you're going to wake up and work hard at it. Nothing is impossible! You should get to the point where anyone else would quit, and you're not going to stop there! No! What are you waiting for? Do it! Just do it! Yes you can! Just do it! If you're tired of starting over, stop giving up!
 The speaker encourages the audience to take action and pursue their dreams. They emphasize that nothing is impossible and that the only way to make their dreams come true is to work hard and not give up. They urge the audience to act now and not wait any longer.

> Finished chain.


' The speaker encourages the audience to take action and pursue their dreams. They emphasize that nothing is impossible and that the only way to make their dreams come true is to work hard and not give up. They urge the audience to act now and not wait any longer.'

# Translate
To perform a translation, we just make few changes to the above workflow.

### New AudioChain with output key set to "translation"

In [6]:
audio_chain = AudioChain(audio_model=audio_model, output_key="translate")

### New SimpleSequentialChain with the new AudioChain and run

In [7]:
translation_to_summary_chain = SimpleSequentialChain(
    chains=[audio_chain, summary_chain], verbose=True
)

In [8]:
audio_path = "example_data/pollo.mp3"
translation_to_summary_chain.run(audio_path)



> Entering new SimpleSequentialChain chain...
I wanted a chicken! Ah, this is the chicken! I like this, I hadn't seen it before! A CHICKEN! I'm afraid of getting my hands dirty and not playing the guitar anymore, otherwise I would break it. Look, a chicken, really, beautiful!
 The speaker is excited to have found a chicken, noting that they have not seen it before. They express fear at getting their hands dirty, as it could prevent them from playing the guitar. Finally, they express admiration for the chicken's beauty.

> Finished chain.


" The speaker is excited to have found a chicken, noting that they have not seen it before. They express fear at getting their hands dirty, as it could prevent them from playing the guitar. Finally, they express admiration for the chicken's beauty."